In [ ]:
'''
# This model trains a CNN using Sequential model in Keras.
# The CNN is a Deep CNN with 2 Convolutional Layers followed by a Artificial Neural Network with 2 hiden layers of
# 128 and 256 nodes in each with ReLU activation function followed by an Softmax output layer
'''

In [1]:
# Importing Keras and Tensorflow modules

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image as img
from keras.models import load_model
from keras.callbacks import History, ModelCheckpoint
import os.path
import numpy as np
import matplotlib.pyplot as plt
import datetime
np.random.seed(2)

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Initilize the CNN

classifier = Sequential()

In [3]:
# Step 1 - Convolution

classifier.add(Conv2D(32, (3, 3), input_shape = (140, 140, 3), activation = 'relu'))
#classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 2 - Add 2nd and 3rd Convolution Layer making it Deep followed by a Pooling Layer

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))


classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
#classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))

In [4]:
# Step 3 - Flattening

classifier.add(Flatten())

In [5]:
# Step 4 - Fully Connected Neural Network

# Hidden Layer 1 - Activation Function RELU
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 3, activation = 'softmax'))

In [6]:
# Compile the CNN
# Categorical Crossentropy - to classify between multiple classes of images
classifier.compile(optimizer = 'adam' , loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [7]:
# Image Augmentation and Training Section

# Image Augmentation to prevent Overfitting (Applying random transformation on images to train set.ie. 
# scalling, rotating and streching)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=0.3,
        width_shift_range=0.3
        )

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# Load the training dataset folder
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(140, 140),
        batch_size=45,
        class_mode='categorical')

Found 9000 images belonging to 3 classes.


In [9]:
# Load the test data set folder
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(140, 140),
        batch_size=45,
        class_mode='categorical')

Found 3000 images belonging to 3 classes.


In [10]:
# Get the accuracy and loss data to plot the graph

history = History()
checkpoint = ModelCheckpoint(filepath='models_backups/' + str(str(datetime.datetime.now().minute) + "-" + str(datetime.datetime.now().second)), monitor='val_loss',verbose=0, mode='auto', period=1)

In [11]:
print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 138, 138, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 136, 136, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 68, 68, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 68, 68, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 66, 66, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)       0         
__________

In [12]:
#Fit the clasifier on the CNN data
if(os.path.isfile('my_model.h5') == False):
    classifier.fit_generator(
            training_set,
            steps_per_epoch=9000,
            epochs=3,
            validation_data=test_set,
            validation_steps=3000,
            callbacks = [history, checkpoint]
    )
    # Save the generated model to my_model.h5
    classifier.save('my_model.h5')
else:
    classifier = load_model('my_model.h5')

Epoch 1/3
 197/9000 [..............................] - ETA: 5497s - loss: 0.9008 - acc: 0.5585

KeyboardInterrupt: 

In [ ]:
# Returns the labels for the classes according to the folder structre of clases
def get_labels_for_clases():
    #return ['car', 'cat', 'dog']
    return ['car' ,'cat', 'dog']

# Run prediction for a single image
def predict_for_single_image(image):
    #lable the images according the folder structure
    
    lables = get_labels_for_clases()
    out = classifier.predict_classes(image)
    print("Prediction index : ", out)
    print("Prediction : ", lables[out[0]])
    #print(classifier.summary())

# Run Prediction for image and give the output as presentatges for each class similarities
def predict_probabilities_for_classes(classifier ,image):
    labels = get_labels_for_clases()
    probabilities = classifier.predict(test_image)
    print(probabilities)
    # Expand two arrays to relevent class structure
    probabilities  = [(format(x * 100, '.2f') + "%") for x in probabilities[0]]

    print(list(zip(labels, probabilities)))
    
          
# Plot the graphs
def plot_graphs_on_data(history):
    
    # Plot Accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epocs')
    plt.legend(['Train Data', 'Test Data'], loc = 'upper left')
    plt.show()

    #Plot Loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epocs')
    plt.legend(['Train Data', 'Test Data'], loc = 'upper left')
    plt.show()

In [ ]:
# Draw the Graph for the predicted Results
# use this only after training.
plot_graphs_on_data(history)

In [ ]:
image = img.load_img('custom_test/dog5.jpg', target_size=(128, 128))
test_image = img.img_to_array(image)
test_image = np.expand_dims(test_image, axis=0)

print(training_set.class_indices)
predict_probabilities_for_classes(classifier, image)